In [1]:
import os
\
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["TORCH_USE_CUDA_DSA"] = "1"


In [2]:
%%time
import lib.utils.utils as utils
from run import get_images
#import lib.config as config
from lib.model.base_model import BaseModel
#import lib.roi as roi
#import lib.split_pages as spages
from lib.utils.promptLoader import PromptLoader

CPU times: user 302 ms, sys: 324 ms, total: 625 ms
Wall time: 1min 32s


In [3]:
image_path = "images/cropped/The_Lightfoot_Herbarium_05_cropped_2.jpg"
image_dir = "images/cropped/"
default_yaml = "./prompts/default_v2.yaml"
conversation = None
if default_yaml is None:
     conversation = [dict(role="user", content=[dict(type="image"), 
                    dict(type="text", 
                         text="Extract the text from both columns in the image, preserving the structure and formatting, ensure no grammatical correction is performed.")])]

In [4]:
print(image_path)

images/cropped/The_Lightfoot_Herbarium_05_cropped_2.jpg


In [5]:
images = sorted(utils.load_images(image_dir))[:10]

In [6]:
images

['images/cropped/The_Lightfoot_Herbarium_05_cropped_1.jpg',
 'images/cropped/The_Lightfoot_Herbarium_05_cropped_2.jpg',
 'images/cropped/The_Lightfoot_Herbarium_06_cropped_1.jpg',
 'images/cropped/The_Lightfoot_Herbarium_06_cropped_2.jpg',
 'images/cropped/The_Lightfoot_Herbarium_07_cropped_1.jpg',
 'images/cropped/The_Lightfoot_Herbarium_07_cropped_2.jpg',
 'images/cropped/The_Lightfoot_Herbarium_08_cropped_1.jpg',
 'images/cropped/The_Lightfoot_Herbarium_08_cropped_2.jpg',
 'images/cropped/The_Lightfoot_Herbarium_09_cropped_1.jpg',
 'images/cropped/The_Lightfoot_Herbarium_09_cropped_2.jpg']

In [7]:
%%time
model = BaseModel("qwen_model", prompt=default_yaml, conversation=conversation, max_new_tokens = 100000, batch_size=1, temperature=0.4)

/mnt/shared/scratch/ikarunak/apps/conda/envs/lightfootcat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/shared/scratch/ikarunak/apps/conda/envs/lightfootcat/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/mnt/shared/scratch/ikarunak/apps/conda/envs/lightfootcat/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `

CPU times: user 3.9 s, sys: 17.8 s, total: 21.7 s
Wall time: 14min 29s


In [8]:
model.conversation

[{'role': 'system',
  'content': 'You are an expert in transcribing text from images. You have knowledge in scientific botany.\n'},
 {'role': 'system',
  'content': '## CONTEXT \nThe images are pages from a botanical catalogue. Each page documents plant species organised by taxonomic hierarchy, starting with the family name at the top level, followed by individual species names, and then folder contents detailing where each specimen was collected. Each page may contain one or two columns of text, and family names are typically written in uppercase, while species names are italicised. Folder contents are listed below each species, with some information potentially continuing from a previous page.\n'},
 {'role': 'system',
  'content': '## GOAL \nYour task is to extract text from the given image and format it into a JSON object that follows the provided structure and specific formatting rules.\n'},
 {'role': 'system',
  'content': '## INSTRUCTIONS \n1) **Parse Text**: Extract all text fro

In [9]:
%%time
outputs = model(images, save=False, debug=True)

Model: qwen_model | Batch Size: 1, Max Tokens: 100000, Temperature: 0.4
Batching Images...
>>> Batch 1 starting...
Extracting text from image
	Processing text prompts...
	Reading Images (If available)...
	Processing inputs...
	Moving inputs to gpu...
	Performing inference...
	Inference Finished
	Seperating Ids...
	Decoding Ids...
	Joining Outputs...


TypeError: can only concatenate list (not "int") to list

In [ ]:
image_name, out = outputs[3]

In [ ]:
print(image_name)
print("="*10)
print(out)

In [ ]:
final_message = "\n\n".join([message for _, message in outputs])
final_message += "\n\nMonocotyledone\n\n Sup"
final_message += "\n\nAlgae\n\n Hello World"
final_message += "\n\nMonocotyledone\n\n Sup"

In [ ]:
print(final_message)

In [ ]:
import re
def clean_message(message):

    result = re.sub(r"\*\*(.+?)\*\*", r"\1", message, flags=re.MULTILINE) # Remove any markdown (bold) on strings
    result = re.sub(r"```", "", result, flags=re.MULTILINE) # Remove any markdown
    result = re.sub(r"^Catalogue|catalogue$", "", result, flags=re.MULTILINE) # Remove Catalogue/catalogue
    result = re.sub(r"^[0-9]+$", "", result, flags=re.MULTILINE) # Remove page numbers
    result = re.sub(r"^John Lightfoot$", "", result, flags=re.MULTILINE) # Remove John Lightfoot name if on its own
    result = re.sub(r"\n{3,}", "", result)
    
    return result

def split_division(message, divisions=["Dicotyledones", "Monocotyledone", "Pteridophyta", "Hepaticae", "Algae"]):

    if divisions is None:
        return [("MAIN", message)]
    
    division_str = "|".join(divisions)
    regex = re.compile(f"({division_str})")
    result = re.split(regex, message)

    result = list(filter(None,result))

    return list(zip(result[::2], result[1::2]))

def split_family(message):

    regex = re.compile("\n\n(?=[A-Z ]+\n)")

    result = re.split(regex, message)

    return list(filter(None,result))

def convertToTextBlocks(message, divisions=["Dicotyledones", "Monocotyledone", "Pteridophyta", "Hepaticae", "Algae"]):

    cleaned_message = clean_message(message)

    division_splits = split_division(cleaned_message, divisions)

    splits = {}

    for division, division_text in division_splits:

        if division in splits:
            splits[division].extend(split_family(division_text))
        else:
            splits[division] = split_family(division_text)
    
    return splits


In [ ]:
result = convertToTextBlocks(final_message)

In [ ]:
result

In [ ]:
split_result

In [ ]:
print(family_split[0][9])

In [ ]:
print(result)

In [ ]:
import math

In [ ]:
math.ceil(0.4578)